# Condition Variables

We often need to have a thread wait for some condition to be true before it can continue executing.

One thread might need to wait for another thread to complete some action before it can continue with its execution.

A similar (but not exact) example is a parent waiting for a child.

We accomplish this with **condition variables**.

A **condition variable** is a queue of waiting threads.

We always need a lock associated with a condition variable, and we have two concurrency functions we use to interact with condition variables:

- `wait(cond_t *cv, lock_t *mutex)`
    - it is assumed that the thread calling wait holds the lock when it is called.
    - adds the thread to the CV queue, puts the caller to sleep, releasing the lock.
    - when woken, `wait()` then reaquires the lock and returns, allowing this thread to continue
- `signal(cond_t *cv)`
    - signals that one thread waiting on the condition variable can wake up and execute



# Example: Implementing Join

We'll implement the ability to have a parent wait for a child.

```C
int done  = 0;
pthread_mutex_t m = PTHREAD_MUTEX_INITIALIZER;
pthread_cond_t c  = PTHREAD_COND_INITIALIZER;

void thr_exit() {
    Pthread_mutex_lock(&m);
    done = 1;
    Pthread_cond_signal(&c);
    Pthread_mutex_unlock(&m);
}

void *child(void *arg) {
    printf("child\n");
    thr_exit();
    return NULL;
}

void thr_join() {
    Pthread_mutex_lock(&m);
    while (done == 0)
        Pthread_cond_wait(&c, &m);
    Pthread_mutex_unlock(&m);
}

int main(int argc, char *argv[]) {
    printf("parent: begin\n");
    pthread_t p;
    Pthread_create(&p, NULL, child, NULL);
    thr_join();
    printf("parent: end\n");
    return 0;
}
```

Do we need the `done` variable?

```C
void thr_exit() {
    Pthread_mutex_lock(&m);
    Pthread_cond_signal(&c);
    Pthread_mutex_unlock(&m);
}

void thr_join() {
    Pthread_mutex_lock(&m);
    Pthread_cond_wait(&c, &m);
    Pthread_mutex_unlock(&m);
}
```

## Condition Variables Best Practices

1) Always grab the lock before calling on wait and release it ASAP afterwards.
2) Always recheck the condition after calling on wait.
    - the state of the condition might have changed between being signaled and reacquiring the lock within `wait()`
3) Always grab the lock before `signal()` and release it ASAP afterwards.

# The Producer/Consumer Model

The producer adds information to some shared buffer.

The consumer retrieves information from the shared buffer.

## Practical Example: A Web Server!!

The producer accepts and adds web requests to a shared queue.

Consumers remove, process, and respond to the web requests from the shared queue.

### Condition Variables

Condition variables are used to ensure that there are web requests on the queue for the consumers to consume (otherwise consumers `wait()` for requests to be added to the queue).

A condition variable is also used to ensure that the queue is not full.

If the queue is empty, consumers will `wait()` until signaled by producers to continue.

If the queue is full, the producers must `wait()` until signaled by consumers to continue.

## A second example: Shell Pipes

The shell pipe uses the producer/consumer model.

`cat source.c | wc -l`

Pipe the output of `cat source.c` into the input of `wc -l`

`wc` must wait until there is input in its buffer before it can start executing.

